In [1]:
import pandas as pd
import numpy as np
import os
import re
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from Tusc5IntegralUtils import *
from Tusc5ImageUtils import *

[2024-08-13 19:58:15,535][INFO]: Connecting Raphael@vfsmdatajoint01.fsm.northwestern.edu:3306
[2024-08-13 19:58:16,065][INFO]: Connected Raphael@vfsmdatajoint01.fsm.northwestern.edu:3306
c:\Users\Luis\anaconda3\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing Data

In [2]:
# '''
# Grabs the fluoresence value from each cell from 'ImageAnalysis folder'

# >>> Need to change directory
# '''


# # Initial Data Import

# directory = '/Users/raphaeltinio/LabAnalysis/ImageAnalysis' # Needs to get to the /ImageAnalysis folder

# stack_names = [f for f in os.listdir(directory) if f != '.DS_Store']

# Cell_Results = pd.DataFrame(columns = ['File', 'Cell', 'Stain', 'X_vals', 'Y_vals'])

# for fname in stack_names:
#     result_directory = directory + "/" + fname + "/results"
    
#     csv_files = [f for f in os.listdir(result_directory) if f.endswith('.csv')] #incase there is a none .csv file for some reason

#     # Grabs number of cells in an image
#     ncells = np.unique([re.search(r'(\d+)(?=\.csv$)', s).group(1) if re.search(r'(\d+)(?=\.csv$)', s) else None for s in csv_files])

#     # Cell Counter
#     if len(Cell_Results) == 0:
#         n_add = 0

#     else:
#         n_add = Cell_Results.Cell.max() + 1 # counts the number of cells after an addition of a z-stack dataframe, so that it keeps counting up 

#     for n in ncells:
#         unique_cell = [f for f in csv_files if "cell" + n + '.csv' in f] # Grabs list of specific cell

#         cell_number = int(n) + n_add

#         for csv in unique_cell:

#             stain_number = csv.split('_')[0]
#             #stain_number = re.findall(r'\d+$', stain_number)[0]

#             single_csv = pd.read_csv(result_directory + '/' + csv, usecols = [1,2])

#             temp_frame = pd.DataFrame({
#                 'File': fname,
#                 'Cell': cell_number,
#                 'Stain': stain_number,
#                 'X_vals': [single_csv['X'].to_list()],
#                 'Y_vals': [single_csv['Y'].to_list()]})

#             Cell_Results = pd.concat([Cell_Results, temp_frame])


# Cell_Results = Cell_Results.reset_index(drop = True)
# Cell_Results['Cell'] = Cell_Results['Cell'].astype('int')
# Cell_Results = Cell_Results.sort_values('Cell')

# # Extracting the first four integers and the following letter into separate columns ({DJID}{Eye})
# Cell_Results['DJID'] = Cell_Results['File'].str.extract(r'(\d{4})').astype(int)
# Cell_Results['Eye'] = Cell_Results['File'].str.extract(r'(\d{4})([A-Za-z])')[1]

# #Renaming Glut1 to GluT1
# Cell_Results['Stain'] = Cell_Results['Stain'].apply(lambda x: 'GluT1' if x == 'Glut1' else x)

# Cell_Results.to_csv('Cell_Results.csv', index = False)

In [3]:
'''
Accepts data from the new image analysis
'''

Cell_Results = pd.read_csv('all_data.csv')
Cell_Results['X_vals'] = Cell_Results['X_vals'].apply(ast.literal_eval) # Makes it so cells containing lists are identifiable as lists by pandas
Cell_Results['Y_vals'] = Cell_Results['Y_vals'].apply(ast.literal_eval)

## DJID_GEN_Eye Merge

In [4]:
'''
'DJID_GEN_EYES.csv' contains DJIDs which connect DJIDs to genotype and which eye was in the experimental or control group

This code chunk merges that dataframe with Cell_Results

'''

Recorded_Info = pd.read_csv('DJID_GEN_EYES.csv')
merged_df = pd.merge(Cell_Results, Recorded_Info[['DJID', 'Genotype', 'Experimental Eye', 'Control Eye']], on='DJID', how='left')

merged_df.loc[merged_df['Eye'] == merged_df['Experimental Eye'], 'Group_Type'] = 'Experimental'

merged_df.loc[merged_df['Eye'] == merged_df['Control Eye'], 'Group_Type'] = 'Control'
Cell_Results = merged_df.drop(columns=['Experimental Eye', 'Control Eye'])

Cell_Results.head()

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Group_Type
0,"(91001, 94476, 97702, 99241, 106455, 108952, 1...",0,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental
1,"(25935, 27691, 32102, 33195, 41392, 45646, 508...",0,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental
2,"(164618, 200794, 266470, 281358, 314684, 30236...",0,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental
3,"(9783, 11396, 14458, 15066, 18389, 19341, 1996...",0,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental
4,"(46406, 47492, 49547, 50177, 53376, 55141, 566...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,False,6.785691,False,het,Experimental


## Birth Data Merge


In [5]:
'''
Fetching a table to connect DJID to mouse age
Then merging that table to Cell_Results

'''

# DJ query and fetch
init_query = sln_animal.Animal.proj('dob', 'sex') * sln_animal.AnimalEvent * sln_animal.Deceased
birth_data = init_query.proj('dob', 'sex', 'date').fetch(format = 'frame')

# Reseting index and renaming columns
birth_data = birth_data.reset_index().drop(columns = ['event_id'])
birth_data = birth_data.rename(columns = {'animal_id': 'DJID', 'dob': 'birth', 'date': 'death'})

# Changing variables to date time vars
birth_data['birth'] = pd.to_datetime(birth_data['birth'])
birth_data['death'] = pd.to_datetime(birth_data['death'])

# Calculate the difference in days
birth_data['Age_Days'] = (birth_data['death'] - birth_data['birth']).dt.days

# Convert days to months (approximately, assuming ~30.44 days in a month)
birth_data['Age_Months'] = round(birth_data['Age_Days'] / 30.44,1)

# Merge
Cell_Results = Cell_Results.merge(birth_data, how = 'left', on = 'DJID')
Cell_Results = Cell_Results.drop(columns = ['birth', 'death', 'Age_Days'])
Cell_Results.head()

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Group_Type,sex,Age_Months
0,"(91001, 94476, 97702, 99241, 106455, 108952, 1...",0,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6
1,"(25935, 27691, 32102, 33195, 41392, 45646, 508...",0,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6
2,"(164618, 200794, 266470, 281358, 314684, 30236...",0,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6
3,"(9783, 11396, 14458, 15066, 18389, 19341, 1996...",0,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6
4,"(46406, 47492, 49547, 50177, 53376, 55141, 566...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,False,6.785691,False,het,Experimental,Male,20.6


## Slice Separation and ID generation

In [6]:
'''
1) Adds a column to the dataframe which contains the z-stack slice seperation value

2) Adds a cell id

'''
Cell_Results['Slice_Seperation'] = Cell_Results['X_vals'].apply(lambda x: x[1]-x[0])

Cell_Results['Cell'] = Cell_Results.groupby(['file_name', 'mask_id']).ngroup()

## Filters

In [7]:
'''
Goes through each cell in Cell_Results and calculates peaks

Prominence = 25 was chosen as it experimentally retained the most number of good cells.

'''

# Peak Filter

print(f'Original Number of Cells: {len(Cell_Results)/4}')

remove_list = []

for cell_number in Cell_Results['Cell'].unique():
    
    CR_Query = Cell_Results.query('Cell == @cell_number').copy()

    slice_seperation = CR_Query.iloc[0]['Slice_Seperation']

    WGA_min_distance = int(3 / slice_seperation) # peaks must be 4 microns away from eachother
    DAPI_min_distance = int(10 / slice_seperation)
    
    index_max = len(CR_Query.iloc[0]['X_vals'])

    WGA_Y_vals = CR_Query.loc[CR_Query['Stain'] == 'WGA']['Y_vals'].iloc[0]
    DAPI_Y_vals = CR_Query.loc[CR_Query['Stain'] == 'DAPI']['Y_vals'].iloc[0]

    peaks_WGA, _ = find_peaks(WGA_Y_vals, prominence= 25, distance = WGA_min_distance)
    peaks_DAPI, _ = find_peaks(DAPI_Y_vals, prominence= 25, distance = DAPI_min_distance)

    '''
    WGA Filters

    '''
    
    # 1) Remove if less than two peaks
    if len(peaks_WGA) < 2:
        remove_list.append(cell_number)
        continue
    

    if peaks_WGA[1] != peaks_WGA[-1] and WGA_Y_vals[peaks_WGA[-1]] > WGA_Y_vals[peaks_WGA[1]]:
        remove_list.append(cell_number)
        continue

    # '''
    # DAPI Filters
    # '''

    if len(peaks_DAPI) < 1:
        remove_list.append(cell_number)
        continue

    elif len(peaks_DAPI) == 1 and peaks_DAPI[0] > peaks_WGA[0] and peaks_DAPI[0] < peaks_WGA[1]:
        pass

    if peaks_DAPI[0] != peaks_DAPI[-1] and DAPI_Y_vals[peaks_DAPI[-1]] > DAPI_Y_vals[peaks_DAPI[1]]:
        remove_list.append(cell_number)

# Removing undesirable cells from main list
remove_list = np.unique(remove_list)
Cell_Results = Cell_Results[~Cell_Results['Cell'].isin(remove_list)]

print(f'Remaining Cells after Filter: {len(Cell_Results)/4}')
print(f'Cell Numbers Removed {remove_list}, count : {len(remove_list)}')

Original Number of Cells: 1824.0
Remaining Cells after Filter: 1786.0
Cell Numbers Removed [   2   13   28   29   32   49  168  171  176  436  445  447  457  613
  627  671  689  704  705  748  752  908  958  985 1047 1082 1084 1108
 1239 1427 1706 1760 1766 1768 1772 1779 1792 1807], count : 38


## eGFP Finder (Retired)

In [8]:
# eGFP finder
eGFP_filtered = []
remove_from_eGFP = []

for cell_number in Cell_Results['Cell'].unique():

    CR_Query = Cell_Results.query('Cell == @cell_number').copy()

    slice_seperation = CR_Query.iloc[0]['Slice_Seperation']
    DAPI_min_distance = int(10 / slice_seperation)

    DAPI_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'DAPI']['Y_vals'].iloc[0], prominence= 25, distance = DAPI_min_distance)
    DAPI_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'DAPI', 'Y_vals'].iloc[0])[DAPI_idx[0]]

    # eGFP peak needs to be at least half of DAPI peak to be considered eGFP positive
    eGFP_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'eGFP']['Y_vals'].iloc[0], prominence= 100, distance = 10, height = int(DAPI_peak_val/2))
    eGFP_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'eGFP', 'Y_vals'].iloc[0])[eGFP_idx]

    WGA_idx, _ = find_peaks(CR_Query.loc[CR_Query['Stain'] == 'WGA']['Y_vals'].iloc[0], prominence= 25)
    WGA_peak_val = np.array(CR_Query.loc[CR_Query['Stain'] == 'WGA', 'Y_vals'].iloc[0])[WGA_idx]

    if len(eGFP_peak_val) == 1:

        # If eGFP peak is not within WGA peaks, cell is not considered eGFP positive
        if eGFP_idx < WGA_idx[0] or eGFP_idx > WGA_idx[1]:
            remove_from_eGFP.append(cell_number)
        
        # If eGFP is not at least 1/2 the size of 
        if eGFP_peak_val > WGA_peak_val[1]/2:
            eGFP_filtered.append(cell_number)


eGFP_filtered = list(set(eGFP_filtered))
eGFP_filtered = sorted([i for i in eGFP_filtered if i not in remove_from_eGFP])

print(f'Cells with eGFP: {eGFP_filtered}, count: {len(eGFP_filtered)}')

# Labeling Cells T5 Positive or Negative
Cell_Results['T5'] = 'Negative'
Cell_Results.loc[Cell_Results['Cell'].isin(eGFP_filtered), 'T5'] = 'Positive'

Cells with eGFP: [38, 40, 53, 56, 57, 66, 102, 104, 114, 117, 124, 143, 152, 181, 182, 215, 223, 232, 270, 274, 281, 290, 292, 315, 327, 329, 346, 358, 368, 369, 400, 401, 408, 411, 416, 420, 460, 486, 488, 489, 492, 494, 500, 502, 511, 513, 517, 530, 539, 554, 564, 578, 580, 593, 596, 608, 669, 759, 787, 788, 792, 813, 843, 858, 868, 883, 888, 897, 898, 902, 906, 911, 913, 916, 923, 926, 927, 933, 935, 953, 954, 955, 1059, 1062, 1064, 1068, 1074, 1080, 1098, 1110, 1138, 1149, 1153, 1169, 1172, 1175, 1189, 1339, 1364, 1367, 1368, 1374, 1375, 1386, 1400, 1417, 1419, 1430, 1438, 1443, 1475, 1478, 1480, 1486, 1490, 1491, 1494, 1504, 1546, 1547, 1548, 1558, 1566, 1570, 1572, 1614, 1624, 1696, 1699, 1782, 1790, 1798, 1812, 1816, 1819, 1822], count: 136


In [9]:
# Comparison between
positive_old_identifier = np.array(eGFP_filtered)
positive_new_identifier = Cell_Results.loc[Cell_Results['eGFP_Value'] == True].Cell.unique()
print('Number of cells in old eGFP finder', len(positive_old_identifier))
print('Number of cells in new eGFP finder', len(positive_new_identifier))
print('Percent of old cells from old eGFP finder in new eGFP finder:', np.isin(positive_old_identifier, positive_new_identifier).sum()/ len(np.isin(positive_old_identifier, positive_new_identifier)) * 100, '%')



Number of cells in old eGFP finder 136
Number of cells in new eGFP finder 281
Percent of old cells from old eGFP finder in new eGFP finder: 88.23529411764706 %


In [10]:
Cell_Results

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Group_Type,sex,Age_Months,Slice_Seperation,Cell,T5
0,"(91001, 94476, 97702, 99241, 106455, 108952, 1...",0,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
1,"(25935, 27691, 32102, 33195, 41392, 45646, 508...",0,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
2,"(164618, 200794, 266470, 281358, 314684, 30236...",0,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
3,"(9783, 11396, 14458, 15066, 18389, 19341, 1996...",0,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
4,"(46406, 47492, 49547, 50177, 53376, 55141, 566...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,False,6.785691,False,het,Experimental,Male,20.6,0.15,1,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,"(2077, 2423, 3057, 3600, 4853, 5987, 9231, 153...",68,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,False,4.000467,False,homo,Experimental,Female,3.3,0.15,1822,Positive
7292,"(26872, 27196, 27099, 27953, 28309, 29354, 304...",69,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative
7293,"(4474, 4442, 4517, 4525, 4724, 4783, 5122, 585...",69,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative
7294,"(11673, 12574, 14052, 16082, 20863, 26054, 410...",69,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative


## Summary of Cell_Results

In [11]:
'''
Sanity Check


'''

print('Genotype', Cell_Results['Genotype'].unique())
print('T5 Labelling', Cell_Results['Stain'].unique())
print('eGFP Types', Cell_Results['eGFP_Value'].unique())

print('\n')
print('###'*12)
print('\n')

'''
Summary of mouse information
'''

def extract_before_underscore(input_string):
    parts = input_string.split("_", 1)
    if len(parts) > 1:
        return parts[0]
    
eyes = Cell_Results['file_name'].apply(extract_before_underscore).unique()

def extract_first_four_integers(input_string):
    # Find all substrings of consecutive digits
    integers = re.findall(r'\d+', input_string)
    
    # Convert the found substrings into integers
    integers = [int(i) for i in integers]
    
    # Return the first four integers
    return integers[:4][0]

animals = list(map(extract_first_four_integers, eyes))
animals = pd.Series(animals).unique() # Extracting the unqiue values

left = 0
right = 0

for i in eyes:
    if 'R' in i:
        right += 1
    if 'L' in i:
        left +=1

positive_count = Cell_Results['eGFP_Value'].value_counts()[True]/4
total = (len(Cell_Results))/4


###
print(f'Number of mice: {len(animals)}')
print(f'Mice DJIDs: {animals}')
print(f'Number of eyes: {len(eyes)}, Left = {left} R = {right}')

###
print('\n')
print('###'*12)
print('\n')

###
print('Number of Cells', Cell_Results['Cell'].count()/4)
print('eGFP Positive', (Cell_Results.groupby('eGFP_Value').size()/4)[True])
print('eGFP Negative', (Cell_Results.groupby('eGFP_Value').size()/4)[False])
print('Positive over Total Percentage', positive_count/total*100)

Genotype ['het' 'homo']
T5 Labelling ['DAPI' 'eGFP' 'WGA' 'GluT1']
eGFP Types [ True False]


####################################


Number of mice: 6
Mice DJIDs: [2006 2007 3101 3201 3203 3318]
Number of eyes: 10, Left = 5 R = 5


####################################


Number of Cells 1786.0
eGFP Positive 281.0
eGFP Negative 1505.0
Positive over Total Percentage 15.733482642777155


In [12]:
Cell_Results.columns

Index(['Y_vals', 'mask_id', 'Stain', 'X_vals', 'file_name', 'DJID', 'Eye',
       'eGFP_Value', 'eGFP_Raw_Intensity', 'in_rip', 'Genotype', 'Group_Type',
       'sex', 'Age_Months', 'Slice_Seperation', 'Cell', 'T5'],
      dtype='object')

In [13]:
Cell_Results.groupby(['Genotype','DJID','sex'])['Age_Months'].unique()

Genotype  DJID  sex   
het       2006  Male      [20.6]
          2007  Male      [20.9]
homo      3101  Male       [5.2]
          3201  Male       [4.3]
          3203  Female     [4.5]
          3318  Female     [3.3]
Name: Age_Months, dtype: object

# Calculations

## Defining Functions

In [14]:
'''
Function: Calculating WGA Peaks

Creates columns: WGA_Middle_Indices, Length (um)

'''

def WGA_Peaks_Finder(dataframe, prom_val:int = 25):

    # Function to find WGA peaks for each cell (via .groupby)
    def find_peaks_in_cell(cell_df):
        # Filter rows where Stain is 'WGA'
        df_WGA = cell_df.loc[cell_df['Stain'] == 'WGA']
        
        slice_seperation = df_WGA.iloc[0]['Slice_Seperation']
        WGA_min_distance = int(3 / slice_seperation) 
        WGA_peak_indicies = find_peaks(df_WGA['Y_vals'].iloc[0], prominence = prom_val, distance = WGA_min_distance)[0]


        return WGA_peak_indicies

    # Apply the peak finder function across cells grouped by 'Cell'
    grouped = dataframe.groupby('Cell')
    slice_separation = grouped['Slice_Seperation'].first().iloc[0]

    grouped_WGA_indices = grouped.apply(find_peaks_in_cell).reset_index().rename(columns={0: 'WGA_Middle_Indices'})    # Reset index to flatten the DataFrame and rename



    grouped_WGA_indices['Length'] = grouped_WGA_indices['WGA_Middle_Indices'].apply(lambda x: (x[1] - x[0]) * slice_separation)

    #Merge the peak indices back into the original DataFrame
    result_df = pd.merge(dataframe, grouped_WGA_indices, on='Cell', how='left')

    return result_df


In [15]:
Cell_Results

,Y_vals,mask_id,Stain,X_vals,file_name,DJID,Eye,eGFP_Value,eGFP_Raw_Intensity,in_rip,Genotype,Group_Type,sex,Age_Months,Slice_Seperation,Cell,T5
0,"(91001, 94476, 97702, 99241, 106455, 108952, 1...",0,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
1,"(25935, 27691, 32102, 33195, 41392, 45646, 508...",0,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
2,"(164618, 200794, 266470, 281358, 314684, 30236...",0,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
3,"(9783, 11396, 14458, 15066, 18389, 19341, 1996...",0,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,True,60.608448,False,het,Experimental,Male,20.6,0.15,0,Negative
4,"(46406, 47492, 49547, 50177, 53376, 55141, 566...",1,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",2006L_GLUT1_647_WGA_594_0001,2006,L,False,6.785691,False,het,Experimental,Male,20.6,0.15,1,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,"(2077, 2423, 3057, 3600, 4853, 5987, 9231, 153...",68,GluT1,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,False,4.000467,False,homo,Experimental,Female,3.3,0.15,1822,Positive
7292,"(26872, 27196, 27099, 27953, 28309, 29354, 304...",69,DAPI,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative
7293,"(4474, 4442, 4517, 4525, 4724, 4783, 5122, 585...",69,eGFP,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative
7294,"(11673, 12574, 14052, 16082, 20863, 26054, 410...",69,WGA,"(0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.7...",3318R_GLUT1_647_WGA_594_0001,3318,R,True,18.464860,False,homo,Experimental,Female,3.3,0.15,1823,Negative


In [16]:
'''
Function: Calculates "Top" and "Bottom" INDICES to be used for integral boundaries

Creates columns: WGA_Top_Indices, WGA_Bottom_Indices
'''

def Top_Bottom_Indices(dataframe, microns_extension: float = 1.5):
    # Grouping / Getting Slice Separation / Calculating peaks
    grouped = dataframe.groupby('Cell')
    slice_separation = grouped['Slice_Seperation'].first()
    max_idx = Cell_Results.groupby('Cell')['X_vals'].first().apply(lambda x: len(x))
    first_peaks = grouped['WGA_Middle_Indices'].apply(lambda x: x.iloc[0] if len(x) > 0 else [np.nan, np.nan])

    # Calculate index offset for each cell based on the slice separation
    index_offset = (microns_extension / slice_separation).fillna(0).astype(int)

    # Separate the peak indices
    l_middle = first_peaks.apply(lambda x: x[0] if len(x) > 0 else np.nan)
    r_middle = first_peaks.apply(lambda x: x[1] if len(x) > 1 else np.nan)

    # Calculate modified indices
    l_top = l_middle - index_offset
    r_bot = r_middle + index_offset

    # Prepare to merge corrections for indices exceeding boundaries
    max_idx = grouped['X_vals'].apply(lambda x: len(x.iloc[0]))

    # Adjust indices to ensure they do not exceed the size of the array
    l_top = np.maximum(l_middle - index_offset, 0)
    r_bot = np.minimum(r_bot, max_idx)

    # Creating DataFrame for merging
    idx_df = pd.DataFrame({
        'Cell': grouped.size().index,  # Assures alignment with group keys
        'WGA_Top_Indices': list(zip(l_top, l_middle)),
        'WGA_Bottom_Indices': list(zip(r_middle, r_bot))
    })

    # Merge adjusted index data back into the original DataFrame
    result_df = pd.merge(dataframe, idx_df, on='Cell', how='left')

    return result_df

In [17]:
'''
Function: Calculates "Top", "Middle", "Bottom" INTEGRALS

Creates columns: Middle_Integral, Top_Integral, Bottom_Integral
'''

def TopMidBot_Integrals(dataframe):
    

    # Allows you to calculate integrals through .apply()
    def integral_calculator(arr1, arr2, integral_section:str = 'Middle'):

        if integral_section == 'Middle':

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]]

        if integral_section == 'Top':

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]] 

        if integral_section == 'Bottom': 

            peak_yvals = np.array(arr1)[arr2[0]:arr2[1]]
        return peak_yvals


    for section in ['Middle', 'Top', 'Bottom']:
        dataframe[f'{section}_Integral'] = dataframe.apply(lambda x: np.sum(integral_calculator(x['Y_vals'], x[f'WGA_{section}_Indices'], integral_section = section)), axis =1)
    
    return dataframe

In [18]:
'''
Function: Calculates "Top", "Bottom" INTEGRAL RATIOS

# Calculating the ratio of Glut1 over WGA (normalization) .5 microns about the WGA peaks

Creates columns: [WGA or GluT1]_[Top or Bot]_Surface_Integral
'''

def Surface_Integrals(dataframe):
    # Initialize a list to store results dataframes for each group
    results = []

    # Group by 'Cell' and process each group
    for (cell, group) in dataframe.groupby('Cell'):
        # Filter for WGA and GluT1 stains
        df_WGA = group.loc[group['Stain'] == 'WGA']
        df_GluT1 = group.loc[group['Stain'] == 'GluT1']


        peak_indices = df_WGA['WGA_Middle_Indices'].iloc[0]
        x_vals = df_WGA['X_vals'].iloc[0]

        # Compute slice separation and radius offset
        slice_separation = group['Slice_Seperation'].iloc[0]
        radius = 0.5
        idx_offset = int(radius / slice_separation)

        # Define borders based on peak indices and radius offset
        top_lborder = max(peak_indices[0] - idx_offset, 0)
        top_rborder = min(peak_indices[0] + idx_offset, len(x_vals))
        bottom_lborder = max(peak_indices[1] - idx_offset, 0)
        bottom_rborder = min(peak_indices[1] + idx_offset, len(x_vals))

        # Calculate integrals for 'GluT1' and 'WGA'
        top_surface_integral_GluT1 = np.sum(df_GluT1.iloc[0]['Y_vals'][top_lborder:top_rborder])
        bottom_surface_integral_GluT1 = np.sum(df_GluT1.iloc[0]['Y_vals'][bottom_lborder:bottom_rborder]) 
        top_surface_integral_WGA = np.sum(df_WGA.iloc[0]['Y_vals'][top_lborder:top_rborder])
        bottom_surface_integral_WGA = np.sum(df_WGA.iloc[0]['Y_vals'][bottom_lborder:bottom_rborder+1])

        # Ratio calculations
        Top_G_over_W = top_surface_integral_GluT1 / top_surface_integral_WGA
        Bottom_G_over_W = bottom_surface_integral_GluT1 / bottom_surface_integral_WGA

        # Append results as a DataFrame to list
        results.append(pd.DataFrame({
            'Cell': [cell],
            'GluT1_Top_Surface_Integral': [top_surface_integral_GluT1],
            'GluT1_Bot_Surface_Integral': [bottom_surface_integral_GluT1],
            'WGA_Top_Surface_Integral': [top_surface_integral_WGA],
            'WGA_Bot_Surface_Integral': [bottom_surface_integral_WGA],
            'Top_Surface_Ratio': [Top_G_over_W],
            'Bot_Surface_Ratio': [Bottom_G_over_W]
        }))

    # Concatenate all results into a single DataFrame
    results_df = pd.concat(results, ignore_index=True)

    # Merge adjusted index data back into the original DataFrame

    dataframe = dataframe.merge(results_df, on='Cell', how='left')
    
    return dataframe

In [19]:
"""
Function: Reshapes columns, so that each group has a column with '{Stain}_{Integral_Type}'. 4 x 3 = 12 columns are added
"""

def Reshape_Integrals(dataframe):

    # Make pivot table
    pivot_df = dataframe.pivot_table(
        index=['Cell'], 
        columns='Stain',
        values=['Middle_Integral', 'Top_Integral', 'Bottom_Integral'],
        aggfunc='first' 
    )

    # Flatten the multi-level column headers and format them
    pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns.values]

    pivot_df.reset_index(inplace=True) # Cell is a column used to merge 

    result_df = pd.merge(dataframe, pivot_df)

    return result_df

In [20]:
Cell_Results_Calcs = Cell_Results.copy()

# These functions find indices
Cell_Results_Calcs = WGA_Peaks_Finder(Cell_Results_Calcs)
Cell_Results_Calcs = Top_Bottom_Indices(Cell_Results_Calcs)

# These functions calculate integrals using the index boundaries
Cell_Results_Calcs = TopMidBot_Integrals(Cell_Results_Calcs)
Cell_Results_Calcs = Reshape_Integrals(Cell_Results_Calcs)
Cell_Results_Calcs = Surface_Integrals(Cell_Results_Calcs)

## Export

In [21]:
'''
Making new dataframe for export
- Keeps certain columns
- Renames columns
- Mapping column categories to new names (e.g. eGFP Positive -> 1)
'''

# Columns to Keep
keep_cols = ['DJID', 'Genotype', 'Eye', 'sex', 'Age_Months', 'eGFP_Value', 'Group_Type', 'in_rip', 'Length',
             'GluT1_Top_Surface_Integral', 'GluT1_Bot_Surface_Integral', 'WGA_Top_Surface_Integral', 'WGA_Bot_Surface_Integral',
             'GluT1_Top_Integral', 'GluT1_Middle_Integral', 'GluT1_Bottom_Integral',
             'WGA_Top_Integral', 'WGA_Middle_Integral', 'WGA_Bottom_Integral']

# Columns to rename
renamed_cols = {'sex': 'Sex',
                'Length': 'Length_um',
                'in_rip': 'In_Rip',
                'Group_Type' :'Experimental_Condition'}

Export_df = Cell_Results_Calcs.groupby('Cell').first().loc[:, keep_cols].reset_index(drop = True)
Export_df = Export_df.rename(columns = renamed_cols)

# Remapping values
Export_df['eGFP_Value'] = Export_df['eGFP_Value'].astype(int)
Export_df['In_Rip'] = Export_df['In_Rip'].astype(int)

Exp_Con_map = {'Experimental': 'Light Flicker', 'Control': 'Dark'}
Export_df['Experimental_Condition'] = Export_df['Experimental_Condition'].map(Exp_Con_map)#.fillna(Export_df['Experimental_Condition'])

# Checking for NA vals
if Export_df.isna().sum().sum() > 0:
    print('There are NA values, check for problems')

Export_df

,DJID,Genotype,Eye,Sex,Age_Months,eGFP_Value,Experimental_Condition,In_Rip,Length_um,GluT1_Top_Surface_Integral,GluT1_Bot_Surface_Integral,WGA_Top_Surface_Integral,WGA_Bot_Surface_Integral,GluT1_Top_Integral,GluT1_Middle_Integral,GluT1_Bottom_Integral,WGA_Top_Integral,WGA_Middle_Integral,WGA_Bottom_Integral
0,2006,het,L,Male,20.6,1,Light Flicker,0,5.70,98619,59933,1643373,695810,50703,559124,67533,913240,4146354,893323
1,2006,het,L,Male,20.6,0,Light Flicker,0,6.15,78437,31174,1003059,605490,38988,427537,41197,506593,2704874,792020
2,2006,het,L,Male,20.6,1,Light Flicker,0,5.55,263247,106138,2668795,1697838,145448,1455354,116580,1633233,7200429,2112107
3,2006,het,L,Male,20.6,1,Light Flicker,0,5.40,662365,209650,3521146,2699267,433298,2684575,239977,2376328,9892941,3475452
4,2006,het,L,Male,20.6,0,Light Flicker,1,4.35,853687,276093,623797,1998485,636068,2764715,379856,546227,4692050,2741074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,3318,homo,R,Female,3.3,0,Light Flicker,0,4.20,522911,345534,2067691,932215,329130,2178877,494825,1416668,5127847,1168601
1782,3318,homo,R,Female,3.3,0,Light Flicker,0,3.45,992121,965720,3701851,3562331,496787,4997897,1330102,2003786,13123019,4660986
1783,3318,homo,R,Female,3.3,0,Light Flicker,0,10.05,166600,29047,546131,166312,117721,836148,39232,412705,2506152,213605
1784,3318,homo,R,Female,3.3,0,Light Flicker,0,4.50,149718,107896,620339,380990,92112,582164,163688,398169,1611158,485323


In [22]:
Export_df.to_csv('processed_data.csv', index = False)